In [5]:
import numpy as np
import bokeh as bk
from bokeh.io import push_notebook
import bokeh.plotting as bkp
import bokeh.models as bkm
from bokeh.io import push_notebook
from ipywidgets import interact
import ipywidgets as widgets
bkp.output_notebook()

Loading BokehJS ...

# Solidification week 5 homework

## Mikko Karkkainen 11740378

### Problem 1. Consider a carbon steel casting (0.8%C) of dimensions 37mm height x 33mm x 33mm. Calculate the cooling rate (G•V)for this casting. Demonstrate that the microstructure of this casting cannot be planar. What microstructure will you expect? Justify your answer.

### Solution:

Average solidification velocity is given by 

$$ V = (v/A)/t_f$$

The final solidification time can be obtained from the Chvorinov equation, presuming a low Biot number (i.e. presuming liquid has uniform temperature). Time-stepping Chvorinov:

$$ T^{n+1} = T^{n} - \frac{A}{v \rho} \frac{T^n-T_m}{\Delta H_f/\Delta T_0 + c} \sqrt{\frac{k_m \rho_m c_m}{\pi t^{n+1}}} \Delta t $$

Let's presume a silica sand mold, to have a minimum thermal gradient.

Data taken for AISI 1008 Carbon steel from Nova database:

<table>
    <tr>
        <th>Material property</th>
        <td>$ \rho $</td>
        <td> $ c $ </td>
        <td> $ k $ </td>
        <td> $ v $</td>
        <td> $ A $</td>
        <td> $ \Delta H_F $ </td>
        <td> $  T_L $ </td>
        <td> $  T_S $ </td>
        <td> $ \Delta T_0 $ </td>
    </tr>
    <tr>
        <th>value</th>
        <td> 7200 </td>
        <td> 687 </td>
        <td> 30 </td>
        <td> 4.029×10^-5 m^3 </td>
        <td> 0.001089 m^2 </td>
        <td> 270 000 </td>
        <td> 1800.241 K </td>
        <td> 1772.705 K </td>
        <td> 27.54 K </td>
    </tr>
</table>

Mold properties:

<table>
    <tr>
        <th>Material property</th>
        <td> $ \rho_m $ </td>
        <td> $ k_m $ </td>
        <td> $ c_m $ </td>
    </tr>
    <tr>
        <th>value</th>
        <td>1600</td>
        <td>0.52</td>
        <td>1170</td>
    </tr>
</table>    

Calculate The solidification time:

In [38]:
Delta_t=1
T_i=1800.241
Delta_T_0=27.54
v=4.029*10**(-5)
A=0.001089
Delta_H_f=270000
rho=7200
c=687
T_L=1800.241
T_S=1772.705

#silica sand mold:

rho_si=1600
k_si=0.52
c_si=1170


#simulation parameters:

step_count=2230

temp_values_si=np.empty(step_count)
temp_values_si[0]=T_i
    
def calc_temp(rho_m,k_m,c_m,temp_values):
    for i in range(1,step_count):
        if (temp_values[i-1]>T_L or temp_values[i-1]<=T_S):
            temp_values[i]=temp_values[i-1]-A/v*(temp_values[i-1]-298.15)/(rho*(c))*np.sqrt((k_m*rho_m*c_m/(np.pi*(Delta_t*i))))*Delta_t
            
        else:
            temp_values[i]=temp_values[i-1]-A/v*(temp_values[i-1]-298.15)/(rho*(Delta_H_f/Delta_T_0 + c))*np.sqrt((k_m*rho_m*c_m/(np.pi*(Delta_t*i))))*Delta_t

calc_temp(rho_si,k_si,c_si,temp_values_si)
        
f=bkp.figure(title="Cooling curve of Fe-0.8%C casting", x_axis_label="time (s)", y_axis_label="Temperature (K)")
time_series=np.linspace(0,step_count*Delta_t,step_count)
time_series_h=np.linspace(Delta_t,step_count*Delta_t,step_count)

f.line(time_series,temp_values_si, color="red", legend="Silica sand mold")

bkp.show(f)

The total solidification time is approximately 2230 seconds. Therefore:

$$ V = (v/A)/t_f = 0.037 m / 2230 s = 1.66 \cdot 10^{-5} m/s $$


Thermal gradient equation from book:

$$ G_T =  (\rho_s \Delta H_f V)/k_s = 7200 \cdot 270000 \cdot 1.66 \cdot 10^{-5} / 30 = 1075.68 K/m  $$

Therefore:
    
$$ \frac{G_T}{V} =  \frac{1075.68}{1.66×10^-5} = 6.48 \cdot 10^7 Ks/m^2 $$

    


Non-steady state criterion for constitutional undercooling:

$$ \frac{G_T}{V} < - \frac{m_L C_L^* (1-k)}{D_L}$$

At the start of solidification, $ C_L^* = C_0 $

Required values:
    
<table>
    <tr>
        <th>Material property</th>
        <td> $ m_L $ </td>
        <td> $ C_0 $ </td>
        <td> $ k $ </td>
        <td> $ D_L $ </td>
    </tr>
    <tr>
        <th>value</th>
        <td>$ -61 $</td>
        <td>$ 0.8 $</td>
        <td>$ 0.28 $</td>
        <td style="width:70px">$ 2\cdot10^{-8} $</td>
    </tr>
</table>   
    

$$ - \frac{m_L C_0 (1-k)}{D_L} = - \frac{-61 \cdot 0.8 (1-0.28)}{2\cdot10^{-8}} = 1.7568 \cdot 10^9 $$

$$ \frac{6.48 \cdot 10^9}{1.7568 \cdot 10^7} \approx 3.7 $$

The thermal gradient is approximately 3.7 times too high for planar solidification. 

### Problem 2. Consider the graph in Fig. 8.16 in the textbook. According to the experimental data in the graph, the coarsening constant is 10-16 m3/s. Using the equations inTable 8.3, find out which assumption gives results closer to the experimentaldata. Provide a table with all the data used in your calculations. Then, using thebest equation, calculate the coarsening constant for an Al-3% Cu and Fe-0.3% C alloys. Comment on the results.

### Problem3. Calculate the change in secondary arm spacing for a Fe-0.9%C alloy for local solidification times between 10-2 and 103 seconds (plot a λ - tf graph). Note:Assume / 2 CL CE Co

### 4. Consider a micro-volume element of an Al-4.5%Cu alloy cooled at constant heatextraction rate ofQ=2108 Jm-3s-1, from an initial temperature of 700°C. Thevolumetric grain density is of 100 grains/mm3. Calculate and graph:
### a. the temperature and solid fraction evolution during the solidification
### b. the evolution of the liquid composition as a function of the fraction solid
### c. the evolution of the SDAS as a function of fraction solid, using Mortensen’s model for dynamic coarsening of spheres. Compare your result with the graph in Fig. 8.16 textbook.
### Note : use as an example application 8.5 textbook